In [1]:
import pandas as pd
import numpy as np
from PIL import Image
from IPython.display import display
import os
import imageio
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms 
import albumentations as A
from albumentations.pytorch import ToTensorV2
import torch
import torch.nn as nn
import torch.optim as optim
import segm

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [19]:
!pip install segmentation-models-pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=8f28e74a6ecc69b84507acc8ac9203bc79a8533334c66111c779251cf392ea26
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60966 sha256=c4ec96054e6acf14ceb8faa9da6370352981d152dff72bc4e4846342f2c45281
  Stored in directory: /root/.cache/pip/wheels/35/cb/a5/8f534c60142835bfc889f9a482e4a67e0b817032d9c6883b64
Successfully built efficientnet-pytorch pretrainedmodels


In [20]:
import segmentation_models_pytorch as smp
import segmentation_models_pytorch.utils.losses

In [2]:
folders = ['dataA','dataB','dataC','dataD','dataE']
image_folder = 'CameraRGB'
mask_folder = 'CameraSeg'

In [3]:
data = []
base = '../input/lyft-udacity-challenge/'
for folder in folders:
    image_path = os.path.join(base,folder+"/"+folder, image_folder)
    mask_path = os.path.join(base,folder+"/"+folder, mask_folder)
    
    image_files = os.listdir(image_path)
    mask_files = os.listdir(mask_path)
    
    for image_file, mask_file in zip(image_files, mask_files):
        image_full_path = os.path.join(image_path, image_file)
        mask_full_path = os.path.join(mask_path, mask_file)
        
        data.append({'id': image_file.split('.')[0], 'imagepath': image_full_path, 'maskpath': mask_full_path})

df = pd.DataFrame(data)

In [25]:
class SDCDataSet(Dataset):
    def __init__(self,imgData):
        self.train = train
        self.imgData = imgData
        self.transform = A.Compose([
                            A.Resize(width=576, height=576),
                            A.HorizontalFlip(p=0.2),
                            A.RandomBrightnessContrast(p=0.2),
                            ToTensorV2(transpose_mask=True)
                        ])
    def __len__(self):
        return len(self.imgData)
    def __getitem__(self,idx):
        img = imageio.v3.imread(self.imgData.iloc[idx].imagepath)
        mask = imageio.v3.imread(self.imgData.iloc[idx].maskpath,mode="L")
        img = img.astype("float32")/255.0
        mask = mask.astype("float32") 
        transformed = self.transform(image=img, mask=mask)
        return transformed['image'],transformed['mask']  

In [26]:
train,test = train_test_split(df,test_size=0.2)

In [27]:
train_ds = SDCDataSet(train)
valid_ds = SDCDataSet(test)

In [28]:
img,mask = train_ds[0]
print(f"img size {img.shape} and mask shape {mask.shape}")

img size torch.Size([3, 576, 576]) and mask shape torch.Size([576, 576])


In [29]:
train_dl = DataLoader(train_ds,batch_size=8)
valid_dl = DataLoader(valid_ds,batch_size=8)

In [30]:
device = torch.device('cuda:0')


In [31]:
model = smp.Unet(in_channels=3,classes=2)

In [32]:
model = model.to(device)

In [36]:
loss = smp.utils.losses.DiceLoss()
metrics = [
    smp.utils.metrics.IoU(threshold=0.5),
]

optimizer = torch.optim.Adam([ 
    dict(params=model.parameters(), lr=0.0001),
])

AttributeError: module 'segmentation_models_pytorch' has no attribute 'utils'

In [ ]:
plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.imshow(img[0].permute(1,2,0))
#plt.title(f'Image {i+1} - Subfolder: {subfolder}')
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(mask[0])  # Display mask in grayscale
#plt.title(f'Mask {i+1} - Subfolder: {subfolder}')
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(predicted_mask_img[:,:],cmap="gray")  # Display mask in grayscale
#plt.title(f'Mask {i+1} - Subfolder: {subfolder}')
plt.axis('off')

plt.show()